In [ ]:
%pip install langchain-community

In [7]:
import logging
import os
from dotenv import load_dotenv
import openai
from pydantic import BaseModel
from fastapi import HTTPException
from langchain.prompts.chat import ChatPromptTemplate, SystemMessagePromptTemplate, HumanMessagePromptTemplate
from langchain_core.prompts import PromptTemplate
from langchain.llms import OpenAI

# Load environment variables
load_dotenv(override=True)

# Environment variables for API keys and model configuration
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
OPENAI_MODEL = os.getenv("OPENAI_MODEL", "gpt-3.5-turbo")

In [8]:
llm = OpenAI(temperature=0.9)

c:\Users\srede\OneDrive\Desktop\Projex\shared-completion-llm\venv\Lib\site-packages\langchain_core\_api\deprecation.py:141: LangChainDeprecationWarning: The class `OpenAI` was deprecated in LangChain 0.0.10 and will be removed in 0.3.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import OpenAI`.
  warn_deprecated(


In [10]:
prompt_template = PromptTemplate.from_template(
    "Tell me a {adjective} joke about {content}."
)


In [11]:
response = llm(prompt_template.format(adjective="funny", content="chickens"))

c:\Users\srede\OneDrive\Desktop\Projex\shared-completion-llm\venv\Lib\site-packages\langchain_core\_api\deprecation.py:141: LangChainDeprecationWarning: The method `BaseLLM.__call__` was deprecated in langchain-core 0.1.7 and will be removed in 1.0. Use invoke instead.
  warn_deprecated(


In [12]:
print(response)



Why did the chicken cross the playground? To get to the other slide!


In [27]:
class ChatOpenAILLM:
    def __init__(self):
        self.logger = logging.getLogger(__name__)
        self._initialize_openai()
        
        # Load templates during initialization
        self.system_message_template = self._load_template("../prompt_templates/system_prompt.txt")
        self.user_message_template = self._load_template("../prompt_templates/user_prompt.txt")

    def _initialize_openai(self):
        try:
            # Initialize OpenAI API key
            openai.api_key = OPENAI_API_KEY
        except Exception as e:
            self.logger.error(f"Failed to initialize OpenAI: {str(e)}")
            raise HTTPException(status_code=500, detail="Failed to initialize OpenAI")

    def create_joke(self, word: str) -> str:
        try:
            # Update user message template with the word
            user_message_template = self.user_message_template.format(word=word)
            prompt = self._generate_prompt(user_message_template)
            self.logger.info(f"Generated prompt: {prompt}")

            response = openai.chat.completions.create(
                model=OPENAI_MODEL,
                messages=[{"role": "system", "content": prompt}],
                max_tokens=100,
                temperature=0.7,
            )
            # Access the content of the response using dot notation
            joke = response.choices[0].message.content.strip()
            self.logger.info(f"Generated joke: {joke}")

            return joke
        except Exception as e:
            self.logger.error(f"Error generating joke: {str(e)}")
            raise HTTPException(status_code=500, detail="Failed to generate joke")

    def _generate_prompt(self, user_message_template: str) -> str:
        chat_template = ChatPromptTemplate.from_messages([
            SystemMessagePromptTemplate.from_template("{system_message}"),
            HumanMessagePromptTemplate.from_template("{user_input}"),
        ])

        messages = chat_template.format_messages(
            system_message=self.system_message_template,
            user_input=user_message_template
        )

        return "\n".join([message.content for message in messages])

    def _load_template(self, filepath: str) -> str:
        try:
            with open(filepath, "r", encoding="utf-8") as file:
                return file.read().strip()
        except Exception as e:
            self.logger.error(f"Failed to load template from {filepath}: {str(e)}")
            raise HTTPException(status_code=500, detail="Failed to load prompt template")

In [28]:
llm = ChatOpenAILLM()

In [30]:
response = llm.create_joke(word='Lagos Agbero')
print(response)

Ah, Lagos Agbero! Dem be like traffic wardens wey no dey wear uniform. You fit dey argue with dem, but if e too much, na your car key dem fit swallow o!
